In [ ]:
import numpy as np
import pandas as pd
import math
from parser_random_exp import parse_results
import matplotlib.pyplot as plt
!pip install -q seaborn tqdm dask[bag]
import seaborn as sns
from matplotlib.ticker import FuncFormatter, MultipleLocator, ScalarFormatter

%load_ext autoreload
%autoreload 2

In [ ]:
def are_exp(values):
    if any(type(v) is bool for v in values):
        return False
    x = sorted(values)
    for i in range(1, len(values)):
        if (x[i-1] * 2 - x[i]) > 0.00000001:
            return False
    return True

def plot_one_to_one_for_params(paths, met='one_es_to_one_gt_accuracy/gan_tokens', figsize=(16,20)):
    if type(paths) is str:
        paths = [paths]
    combined_proc_data = [None] * len(paths)
    params_data = [None] * len(paths)
    for i, path in enumerate(paths):
        data, params = parse_results(path)
        res = data[data['name'] == met]
        best_for_run = res.groupby(['exp_id', '_try', 'name'])['exp_id', '_try', 'name', 'value'].max()
        combined_proc_data[i] = pd.merge(params, best_for_run,left_index=True, right_on='exp_id',  how='outer')
        params_data[i] = params
    
    all_params = list(set([x for p in params_data for x in p.columns.to_list()]))

    fig, axs = plt.subplots(math.ceil(len(all_params) / 2), 2 , figsize=figsize)
    for i, param in enumerate(all_params):
        for exp_num, (proc_data, params) in enumerate(zip(combined_proc_data, params_data)):
            if param not in params.columns.to_list():
                continue
            log_plot: bool = are_exp(list(set(proc_data[param].values)))
            if log_plot:
                axs.flatten()[i].set_xscale('log', basex=2)
                axs.flatten()[i].xaxis.set_major_formatter(ScalarFormatter())
            sns.regplot(
                x=param,
                y='value',
                data=proc_data,
                ax=axs.flatten()[i],
                dropna=True,
                logx=log_plot,
                label=paths[exp_num]
            ).set_title(param.split('.')[-1])
            axs.flatten()[i].legend()


In [ ]:
plot_one_to_one_for_params(
    [
        'runs/2020_05_28/id_protos_gan_without_blanks/',
        'runs/2020_05_21/id_all_as_protos/',
    ],
    figsize=(16,8),
)